In [1]:
import qepy
import numpy as np

In [2]:
try:
    from mpi4py import MPI
    comm=MPI.COMM_WORLD
except:
    comm=None

In [3]:
from qepy.calculator import QEpyCalculator
import ase.io
from ase.io.trajectory import Trajectory
from ase import units
from ase.md.andersen import Andersen
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [4]:
inputfile = 'qe_in.in'
qepy.qepy_mod.qepy_set_stdout('qepy.out')

calc = QEpyCalculator(comm = comm, inputfile = inputfile)
atoms = ase.io.read(inputfile, format='espresso-in')
atoms.set_calculator(calc)

T = 300
MaxwellBoltzmannDistribution(atoms, temperature_K = T, force_temp=True)
dyn = Andersen(atoms, 1.0 * units.fs, temperature_K = T, andersen_prob=0.02)

In [5]:
step = 0
interval = 1

def printenergy(a=atoms):
    global step, interval
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    if a.calc.rank == 0 :
        print("Step={:<8d} Epot={:.5f} Ekin={:.5f} T={:.3f} Etot={:.5f}".format(
                step, epot, ekin, ekin / (1.5 * units.kB), epot + ekin, flush=True))
    step += interval

In [6]:
class WF(object):
    def __init__(self, calc=None, kpt=0, band=3):
        self.calc=calc
        self.kpt=kpt
        self.band=band
        self.wf_prev=None
        self.wf=None
    def __call__(self, i=0, j=0):
        wf = self.calc.get_wave_function(kpt=self.kpt, band=self.band)
        if self.wf is None:
            self.wf = wf
            self.wf_prev = wf
        else:
            self.wf, self.wf_prev = self.wf_prev, self.wf
            self.wf = wf
        result=(self.wf_prev[i]*self.wf[j].conj()).sum()/len(self.wf[j])
        
        if comm:
            result=comm.bcast(result)
            if comm.rank==0:
                print('WF:', result, flush=True)
        else:
            print('WF:', result, flush=True)
        return result

In [7]:
wf=WF(calc=calc, kpt=0, band=[3,4])
traj = Trajectory("md.traj", "w", atoms)
dyn.attach(printenergy, interval=1)
dyn.attach(traj.write, interval=1)
dyn.attach(wf, interval=1)
dyn.run(5)

Step=0        Epot=-154.42176 Ekin=0.03878 T=300.000 Etot=-154.38298
WF: (0.9999999999999996+0j)
Step=1        Epot=-154.42181 Ekin=0.01892 T=146.369 Etot=-154.40289
WF: (-0.999879314023658+5.515031874781374e-19j)
Step=2        Epot=-154.41875 Ekin=0.01612 T=124.730 Etot=-154.40262
WF: (0.9998801724992373-3.1795545470303174e-18j)
Step=3        Epot=-154.41588 Ekin=0.01025 T=79.313 Etot=-154.40563
WF: (0.9998699800195554+1.553622557429328e-18j)
Step=4        Epot=-154.41628 Ekin=0.01044 T=80.732 Etot=-154.40584
WF: (-0.999863637924897-1.2957797098888896e-18j)
Step=5        Epot=-154.41729 Ekin=0.01120 T=86.635 Etot=-154.40609
WF: (0.9998579907329598+1.6036856389738197e-18j)


True